## Import dependencies

In [ ]:
import pandas as pd
import requests
import json

from helper_function.notebook_helpers import show_vehicle_routes, get_minutes_from_datetime
from helper_function.map_helpers import get_map_by_vehicle

from cuopt_thin_client import CuOptServiceClient

Finally, suppose you are working at a company for grocery store fridge installation and maintenance. Once again, you are working with the same grocery stores. Some of the grocery stores are new so they have put in a request to install fridges. Some of the stores already have fridges but request some sort of maintenance. Given input data about stores' service requests and the available fleet of vehicles, it is your job to calculate the route for each vehicle such that all service requests are fulfilled while minimizing vehicles' travel time and cost. In this notebook, we will walk through the data preprocessing steps needed in order to utilize cuOpt for this use case. 

## Read input data from CSV files

For a Dispatch Optimization problem, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)

- Orders
    - Location
    - Start and end time (customer indicated time window)
    - Demand (service type- either install or maintenance)

- Vehicles
    - Name/ID Number
    - Start and end depot name
    - Start and end time (vehicle/driver shift hours)
    - Capacity (in this problem is given in time- how long a driver can work for)
    - Vehicle skills (whether this driver can provide install or maintenance service)



You may have additional features depending on the problem at hand.

In [ ]:
depots_df = pd.read_csv('data/depots_do.csv')
orders_df = pd.read_csv('data/orders_do.csv')
vehicles_df = pd.read_csv('data/vehicles_do.csv')

In [ ]:
n_depots = len(depots_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [ ]:
locations_df = (pd.concat([depots_df[["Name","Longitude","Latitude"]], orders_df[["Name","Longitude","Latitude"]]], ignore_index=True)).reset_index()

# Create cost matrix

The **cost matrix** models the cost between each pair of locations.  It is used by cuOpt to compute the cost of traveling from any location to any other. The cost matrix needs to be a square matrix of dimension equal to the total number of locations which inlcludes both depots and orders. In this Vehicle Routing Problem, our cost metric is travel time. This is cost we want to minimize. 

To build a a cost matrix of live traffic data, we need to use a third party map data provider. In this workflow, the cost matrix will calculate the travel time in minutes between each two pairs of locations which we build using OSRM. 

In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

In [ ]:
def build_travel_time_matrix(df):
    latitude = df.Latitude.to_numpy()
    longitude = df.Longitude.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [ ]:
cost_matrix_df = build_travel_time_matrix(locations_df)

### Set fleet data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- vehicle_locations is a list of the start and end location of the vehicles. For example, a vehicle that starts and ends in depot 1 which is the location at index 0 would have the vehicle location of [0,0]. While each vehicle has an assigned location, in this use case, drivers may start and end their shift wherever they'd like. For example, they might wake up at home in the morning and go directly to their first task. Similarly, at the end of the day, they might finish their last task and go straight home without stopping in their depot. To represent this, we pass an array of booleans for skip_first_trips and skip_last_trips, where the value is True for all vehicles

- capacities is the total amount of time a driver is available for represented in minutes. 

- vehicle_max_times is the maximum length of a shift a driver should work. For example, a driver might be available to work for 9 hours in a day but a shift should not exceed 6 hours, such that the driver will work 6 out of these 9 hours.

- vehicle_time_windows is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

In [ ]:
depot_names_to_indices_dict = {locations_df["Name"].values.tolist()[i]: i for i in range(n_depots)}
vehicle_locations = vehicles_df[["assigned_depot","assigned_depot"]].replace(depot_names_to_indices_dict).values.tolist()

In [ ]:
skip_first_trips = [True]*n_vehicles
drop_return_trips = [True]*n_vehicles

In [ ]:
vehicle_time_windows = pd.concat((vehicles_df['vehicle_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['vehicle_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [ ]:
vehicle_max_times = vehicles_df['max_shift_time'].values.tolist()

### Set task data


Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- task_locations is the locations where customers have requested service. 

- task_time_windows is the list of integer representation of the customer indicated time window in which the service provider can come to deliver the requested service. 

- service_times is the list of the length of time it takes for each service to be fulfilled. We let install service be 60 minutes, and maintenance service be 120 minutes.

- order vehicle match is a list of dictionaries that map which vehicles can provide the service requested in each location. Some vehicles can provide install service, some can provide maintenance service, and some can do both. For a task that has requested install service, we want to assign all the vehicles that can fulfill this type of request.

In [ ]:
task_locations = locations_df.index.tolist()[n_depots:]
task_time_windows = pd.concat((orders_df['order_start_time'].apply(get_minutes_from_datetime).to_frame(), orders_df['order_end_time'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [ ]:
demand_service = orders_df["service"]

In [ ]:
demand_service_time = {"install":60, "maintenance":120}
demand_time = orders_df["service"].replace(demand_service_time).tolist()

In [ ]:
install_tech_ids = vehicles_df['install_service'][vehicles_df['install_service']==1].index.values.tolist()
maintenance_tech_ids = vehicles_df['maintenance_service'][vehicles_df['maintenance_service']==1].index.values.tolist()

In [ ]:
vehicle_match_list = []

for i in range(len(orders_df['service'].index)):
    if orders_df['service'][i] == 'install':
        vehicle_match_list.append({"order_id": i, "vehicle_ids": install_tech_ids})
    if orders_df['service'][i] == 'maintenance':
        vehicle_match_list.append({"order_id": i, "vehicle_ids": maintenance_tech_ids}) 


### Set Solver configuration

Before we send our data to the cuOpt solver, we will add a few several configuration settings.

- the time_limit is the maximum time allotted to find a solution. This depends on the user, who has the flexibility of setting a higher time‑limit for better results. 

- the number of parallel agents (climbers) examining the solution search space.

If the user wants the first solution, then around 2-3 seconds for 2000-4000 climbers are enough. cuOpt solver does not interrupt the initial solution. So if the user specifies a shorter time than it takes for the initial solution, the initial solution is returned when it is computed. Increasing the number of climbers will increase the time it takes to compute the initial solution.
By default, the number of climbers is chosen by considering occupancy of a small GPU and experimented runtime vs number of climbers trade-off (that is, the best result in shortest time). Normally 1024 is a good start.

In [ ]:
# Set the time limit 
# Set number of climbers that will try to search for an optimal routes in parallel

time_limit = 0.1
number_of_climbers = 1024 


## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our dispatch optimization problem. 

In [ ]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "travel_time_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "task_time_windows": task_time_windows,
        "service_times": demand_time,
        "order_vehicle_match": vehicle_match_list,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "skip_first_trips" : skip_first_trips,
        "drop_return_trips": drop_return_trips,
        "vehicle_max_times": vehicle_max_times,
        "vehicle_time_windows": vehicle_time_windows,
    },
    "solver_config": {
        "time_limit": time_limit,
        "number_of_climbers": number_of_climbers
    }
}

## Create a Service Client Instance

Now that we have prepared all of our data, we can establish a connection to the cuOpt service. 

In the cell below, paste your client ID and Secret that you received via email in order to authenticate the connection. 

Here, we create an instance of the cuOpt Service Client to establish a connection. 

In [ ]:
cuopt_client_id = "<YOUR CLIENT ID>"
cuopt_client_secret = "<YOUR CLIENT SECRET>"

cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

## Send data to the cuOpt service and get the routes

When using the Python SDK or microservice, we need to send all the data in to cuOpt in multiple steps or API calls. 

When using the managed service, we send all the data in at once. The Python thin client includes an endpoint that send the data over to the service and returns the service output.

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

# Visualize the routes

In this example, not all vehicles are dispatched. It is possible that vehicle 0 is not dispatched but vehicle 1 is.  

In the drop down menu below, you can select different vehicle ID's to see if they are dispatched. If they are, we print their assigned route on a map. 

Generating a route and map uses third party tools and takes about 30 seconds to run. 


In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=["index"])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)